可以看到DenseNet 里来自跳层的输出不是通过加法（+）而是拼接（concat) l来跟目前层的输出合并。因为是拼接，所以底层的输出会保留的进入上面的所有层。这是为什么叫做“稠密连接" 的原因 。
# 稠密块 （Dense Block) 
我们来定义一个稠密连接块。 DenseNet的卷积块使用ResNet改进版本的BN->Relu->Conv。每个卷积的输出通道数被称为growth_rate,这是因为假设输出为in_channels,而且有layers 层，那么输出的通道数就是in_channels+groth_rate+layers。

In [1]:
from mxnet import nd 
from mxnet.gluon import nn 
def conv_block(channels):
    out=nn.Sequential()
    out.add(
        nn.BatchNorm(),
        nn.Activation('relu'),
        nn.Conv2D(channels,kernel_size=3,padding=1) 
    )
    return out 

class DenseBlock(nn.Block):
    def __init__(self,layers,growth_rate,**kwargs):
        super(DenseBlock,self).__init__(**kwargs)
        self.net=nn.Sequential()
        for i in range(layers):
            self.net.add(conv_block(growth_rate)) 
    def forward(self,X):
        for layer in self.net:
            out=layer(X)
            X=nd.concat(X,out,dim=1) 
        return X 

我们验证下输出通道数是不是符合预期 

In [3]:
dblk=DenseBlock(3,11) 
dblk.initialize() 

x=nd.random.uniform(shape=(4,3,8,8)) 
dblk(x).shape

(4, 36, 8, 8)

# 过度块（Transition Block) 
因为使用拼接的缘故，没经过一次过渡块输出通道数可能会激增。为了控制模型复杂度，这里引入一个过渡块，它不仅把输入的长度减半，同事也使用1X1卷积来改变通道数 

In [4]:
def transition_block(channels):
    out=nn.Sequential() 
    out.add(
        nn.BatchNorm(),
        nn.Activation('relu'),
        nn.Conv2D(channels,kernel_size=1) ,
        nn.AvgPool2D(pool_size=2,strides=2) 
    )
    return out 

验证一下结果 ： 

In [5]:
tblk=transition_block(10) 
tblk.initialize() 
tblk(x).shape

(4, 10, 4, 4)

# DenseNet 
DenseNet 的主体就是交替串联稠密块和过渡块。它使用全局的growth_rate来使得配置更加简单。过渡层每次都会将通道数减半。
下面定义一个121层的DenseNet。

In [6]:
init_channels=64
growth_rate=32
block_layers=[6,12,24,16]
num_classes=10 


def dense_net():
    net=nn.Sequential()
    with net.name_scope():
        # first block 
        net.add(
            nn.Conv2D(init_channels,
                      kernel_size=7,strides=2,padding=3),
            nn.BatchNorm(),
            nn.Activation('relu') ,
            nn.MaxPool2D(pool_size=3,strides=2,padding=1) 
        )
        # dense blocks 
        channels=init_channels 
        for i,layers in enumerate(block_layers):
            net.add(DenseBlock(layers,growth_rate)) 
            channels+=layers* growth_rate 
            if i!=len(block_layers)-1:
        
        # last block 
        net.add( 
            nn.BatchNorm(),
            nn.Activation('relu'),
            nn.AvgPool2D(pool_size=1),
            nn.Flatten(),
            nn.Dense(num_classes) 
        )
    return net 